<a href="https://colab.research.google.com/github/riyad03/EarthSmart/blob/main/sustainable_consumption_and_production_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas import read_csv
import tqdm as tqdm
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error

In [ ]:
df=read_csv('/content/domestic-material-consumption-per-capita (1).csv',header=0)

In [ ]:
df.drop(df.columns[1],inplace=True,axis=1)

In [ ]:
df

,Entity,Code,Year,"12.2.2 - Domestic material consumption per capita, by type of raw material (tonnes) - EN_MAT_DOMCMPC - Total or no breakdown"
0,Afghanistan,AFG,2000,1.17
1,Afghanistan,AFG,2001,1.08
2,Afghanistan,AFG,2002,1.33
3,Afghanistan,AFG,2003,1.35
4,Afghanistan,AFG,2004,1.21
...,...,...,...,...
4413,Zimbabwe,ZWE,2015,5.27
4414,Zimbabwe,ZWE,2016,5.30
4415,Zimbabwe,ZWE,2017,5.40
4416,Zimbabwe,ZWE,2018,5.76


In [ ]:
values=df.values

In [ ]:
data=[]
element='Afghanistan'
val=[]
for i in tqdm.tqdm(range(len(values))):

  val.append(values[i,2])
  if i!=len(values)-1:
    if values[i+1,0]!=element:
      val=np.array(val)
      data.append(val)
      element=values[i+1,0]
      val=[]
  else:
    val=np.array(val)
    data.append(val)


100%|██████████| 4418/4418 [00:00<00:00, 490288.00it/s]


In [ ]:
len(data)

222

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scalelist=[]
for i in range(len(data)):
  scalelist.append(scaler.fit_transform(pd.DataFrame(data[i])))


In [ ]:
scalelist

[array([[0.        ],
        [0.05263158],
        [0.10526316],
        [0.15789474],
        [0.21052632],
        [0.26315789],
        [0.31578947],
        [0.36842105],
        [0.42105263],
        [0.47368421],
        [0.52631579],
        [0.57894737],
        [0.63157895],
        [0.68421053],
        [0.73684211],
        [0.78947368],
        [0.84210526],
        [0.89473684],
        [0.94736842],
        [1.        ]]),
 array([[0.        ],
        [0.05263158],
        [0.10526316],
        [0.15789474],
        [0.21052632],
        [0.26315789],
        [0.31578947],
        [0.36842105],
        [0.42105263],
        [0.47368421],
        [0.52631579],
        [0.57894737],
        [0.63157895],
        [0.68421053],
        [0.73684211],
        [0.78947368],
        [0.84210526],
        [0.89473684],
        [0.94736842],
        [1.        ]]),
 array([[0.        ],
        [0.05263158],
        [0.10526316],
        [0.15789474],
        [0.21052632],
      

In [ ]:
len(scalelist)

222

In [ ]:
def ToSupervised(dt,past_time=-1,future_time=3):
  df=pd.DataFrame(dt)
  if past_time<0:
    past_time=-past_time
  col,names=[],[]
  for i in range(past_time,0,-1):
    col.append(df.shift(i))
    names+=['t -'+str(i)]
  for i in range(0,future_time):
    col.append(df.shift(-i))
    names+=['t +'+str(i)]
  agg=pd.concat(col,axis=1)
  agg.columns=names
  agg.dropna(inplace=True)
  return agg.values


In [ ]:
reframe=[]
for i in range(len(scalelist)):
  reframe.append(ToSupervised(scalelist[i],-3,3))

In [ ]:
reframe

[array([[0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
         0.26315789],
        [0.05263158, 0.10526316, 0.15789474, 0.21052632, 0.26315789,
         0.31578947],
        [0.10526316, 0.15789474, 0.21052632, 0.26315789, 0.31578947,
         0.36842105],
        [0.15789474, 0.21052632, 0.26315789, 0.31578947, 0.36842105,
         0.42105263],
        [0.21052632, 0.26315789, 0.31578947, 0.36842105, 0.42105263,
         0.47368421],
        [0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
         0.52631579],
        [0.31578947, 0.36842105, 0.42105263, 0.47368421, 0.52631579,
         0.57894737],
        [0.36842105, 0.42105263, 0.47368421, 0.52631579, 0.57894737,
         0.63157895],
        [0.42105263, 0.47368421, 0.52631579, 0.57894737, 0.63157895,
         0.68421053],
        [0.47368421, 0.52631579, 0.57894737, 0.63157895, 0.68421053,
         0.73684211],
        [0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
         0.78947368],

In [ ]:
lastel=reframe.pop(-1)

In [ ]:
reframe[0][0]

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789])

In [ ]:
train=reframe[:92]
test=reframe[92:]

In [ ]:
len(test)

130

In [ ]:
t=train[46:]
x=train[:46]

In [ ]:
len(t)

46

In [ ]:
len(x)

46

In [ ]:
train[0].shape

(15, 6)

In [ ]:
len(test)

130

In [ ]:
i=24
print('train num',i)
x=train
t=train[46:]
x=train[:46]
train_x,train_y=x[i][:,:-1],x[i][:,-1:]
val_x,val_y=t[i][:,:-1],t[i][:,-1:]
train_x=train_x.reshape((train_x.shape[0],1,train_x.shape[1]))
val_x=val_x.reshape((val_x.shape[0],1,val_x.shape[1]))
hist=model.fit(train_x,train_y,validation_data=(val_x,val_y),batch_size=28,verbose=1,epochs=20,shuffle=False)
records.append(hist)

train num 24


ValueError: Expected input data to be non-empty.

In [ ]:
x[25]

array([[0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
        0.26315789],
       [0.05263158, 0.10526316, 0.15789474, 0.21052632, 0.26315789,
        0.31578947],
       [0.10526316, 0.15789474, 0.21052632, 0.26315789, 0.31578947,
        0.36842105],
       [0.15789474, 0.21052632, 0.26315789, 0.31578947, 0.36842105,
        0.42105263],
       [0.21052632, 0.26315789, 0.31578947, 0.36842105, 0.42105263,
        0.47368421],
       [0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
        0.52631579],
       [0.31578947, 0.36842105, 0.42105263, 0.47368421, 0.52631579,
        0.57894737],
       [0.36842105, 0.42105263, 0.47368421, 0.52631579, 0.57894737,
        0.63157895],
       [0.42105263, 0.47368421, 0.52631579, 0.57894737, 0.63157895,
        0.68421053],
       [0.47368421, 0.52631579, 0.57894737, 0.63157895, 0.68421053,
        0.73684211],
       [0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
        0.78947368],
       [0.57894737, 0

In [ ]:
# prompt: check if x[24] is empty

if x[23].any():
  print('x[24] is empty')


x[24] is empty


In [ ]:
val_y

array([[0.26315789],
       [0.31578947],
       [0.36842105],
       [0.42105263],
       [0.47368421],
       [0.52631579],
       [0.57894737],
       [0.63157895],
       [0.68421053],
       [0.73684211],
       [0.78947368],
       [0.84210526],
       [0.89473684],
       [0.94736842],
       [1.        ]])

In [ ]:
len(train[:46])

46

In [ ]:
t[24]

array([[0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
        0.26315789],
       [0.05263158, 0.10526316, 0.15789474, 0.21052632, 0.26315789,
        0.31578947],
       [0.10526316, 0.15789474, 0.21052632, 0.26315789, 0.31578947,
        0.36842105],
       [0.15789474, 0.21052632, 0.26315789, 0.31578947, 0.36842105,
        0.42105263],
       [0.21052632, 0.26315789, 0.31578947, 0.36842105, 0.42105263,
        0.47368421],
       [0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
        0.52631579],
       [0.31578947, 0.36842105, 0.42105263, 0.47368421, 0.52631579,
        0.57894737],
       [0.36842105, 0.42105263, 0.47368421, 0.52631579, 0.57894737,
        0.63157895],
       [0.42105263, 0.47368421, 0.52631579, 0.57894737, 0.63157895,
        0.68421053],
       [0.47368421, 0.52631579, 0.57894737, 0.63157895, 0.68421053,
        0.73684211],
       [0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
        0.78947368],
       [0.57894737, 0

In [ ]:
len(train)/2

46.0

In [ ]:
x[14:]

array([[0.73684211, 0.78947368, 0.84210526, 0.89473684, 0.94736842,
        1.        ]])

In [ ]:
train[i][22:]

,t -3,t -2,t -1,t +0,t +1,t +2


In [ ]:
val_x

array([], shape=(0, 5), dtype=float64)

In [ ]:
model = Sequential()
model.add(LSTM(15, input_shape=(1, 5)))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
records=[]
for i in range(46):
  print('train num',i)
  x=train
  t=train[46:]
  x=train[:46]
  if not (x[i].any() and t[i].any()):
    continue
  train_x,train_y=x[i][:,:-1],x[i][:,-1:]
  val_x,val_y=t[i][:,:-1],t[i][:,-1:]
  train_x=train_x.reshape((train_x.shape[0],1,train_x.shape[1]))
  val_x=val_x.reshape((val_x.shape[0],1,val_x.shape[1]))
  hist=model.fit(train_x,train_y,validation_data=(val_x,val_y),batch_size=28,verbose=1,epochs=20,shuffle=False)
  records.append(hist)

train num 0
Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 0.6832 - val_loss: 0.6766
Epoch 2/20
1/1 [==============================] - 0s 77ms/step - loss: 0.6766 - val_loss: 0.6700
Epoch 3/20
1/1 [==============================] - 0s 68ms/step - loss: 0.6700 - val_loss: 0.6635
Epoch 4/20
1/1 [==============================] - 0s 51ms/step - loss: 0.6635 - val_loss: 0.6569
Epoch 5/20
1/1 [==============================] - 0s 70ms/step - loss: 0.6569 - val_loss: 0.6503
Epoch 6/20
1/1 [==============================] - 0s 50ms/step - loss: 0.6503 - val_loss: 0.6437
Epoch 7/20
1/1 [==============================] - 0s 51ms/step - loss: 0.6437 - val_loss: 0.6371
Epoch 8/20
1/1 [==============================] - 0s 46ms/step - loss: 0.6371 - val_loss: 0.6306
Epoch 9/20
1/1 [==============================] - 0s 35ms/step - loss: 0.6306 - val_loss: 0.6240
Epoch 10/20
1/1 [==============================] - 0s 44ms/step - loss: 0.6240 - val_loss: 0.6174
Epoch 11/20
1/1 [==

In [ ]:
rmse_record=[]
for i in range(len(test)):
  t=test
  test_x=t[i][:,:-1]
  test_y=t[i][:,-1]
  test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))
  yhat=model.predict(test_x)
  test_x = test_x.reshape((test_x.shape[0], test_x.shape[2]))
  # invert scaling for forecast
  inv_yhat = np.concatenate((yhat, test_x[:, 1:]), axis=1)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]

  # invert scaling for actual
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = np.concatenate((test_y, test_x[:, 1:]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]

  rmse_record.append(np.sqrt(mean_squared_error(inv_y, inv_yhat)))

1/1 [==============================] - 0s 22ms/step


In [ ]:
rmse_record

[0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.027871647952314626,
 0.02787164

In [ ]:
t=[	7.31,	7.63,7.94,8.39,9.11]
t = np.array(t).reshape(-1, 1)
scaling= StandardScaler()
t=scaler.fit_transform(t)
t=np.array(t)
t= t.reshape((t.shape[1], 1, t.shape[0]))
yhat=model.predict(t)

1/1 [==============================] - 0s 138ms/step


In [ ]:
t = t.reshape((t.shape[0],t.shape[2]))
inv_yhat = np.concatenate((yhat, t[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
inv_yhat

array([8.33576621])

In [ ]:
model.save("sustainable consumption and production.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test for countries

In [ ]:
ent='Afghanistan'
entlist=[ent]
for i in tqdm.tqdm(range(len(df))):
  if(ent != df['Entity'].values[i]):
    ent=df['Entity'].values[i]
    entlist.append(ent)


100%|██████████| 4418/4418 [00:00<00:00, 141558.52it/s]


In [ ]:
from keras import models

renewable= models.load_model("/content/sustainable consumption and production.h5")

In [ ]:
entlistindex=0
newdata=["Entity,Year,(% equivalent primary energy)"]
while(entlistindex<len(entlist)):

  lastdata=[]
  #add 5 last data
  lastdata.append(df[df['Entity']==entlist[entlistindex]].tail(5).values[:,-1].tolist())
  year=2020
  while (year<=2027):
    newtest=lastdata[0]

    #standardize
    newtest=np.array(newtest).reshape(-1,1)
    scaler=MinMaxScaler(feature_range=(0,1))
    newtest=scaler.fit_transform(newtest)
    newtest=np.array(newtest)
    #reshape
    newtest=newtest.reshape(newtest.shape[1],1,newtest.shape[0])
    if(newtest.shape!=(1,1,5)):
      break
    #prediction
    ypred= renewable.predict(newtest)
    #inverse prediction
    newtest = newtest.reshape((newtest.shape[0],newtest.shape[2]))
    inv_yhat = np.concatenate((ypred, newtest[:, 1:]), axis=1)

    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    lastdata[0].append(inv_yhat[0])
    lastdata=[lastdata[0][1:]]

    newdata.append(entlist[entlistindex]+","+str(year)+","+str(inv_yhat[0]))
    print(year," ",entlistindex)
    year=year+1
  entlistindex=entlistindex+1


1/1 [==============================] - 0s 24ms/step
2020   0
1/1 [==============================] - 0s 21ms/step
2021   0
1/1 [==============================] - 0s 21ms/step
2022   0
1/1 [==============================] - 0s 21ms/step
2023   0
1/1 [==============================] - 0s 22ms/step
2024   0
1/1 [==============================] - 0s 22ms/step
2025   0
1/1 [==============================] - 0s 20ms/step
2026   0
1/1 [==============================] - 0s 21ms/step
2027   0
1/1 [==============================] - 0s 21ms/step
2020   1
1/1 [==============================] - 0s 20ms/step
2021   1
1/1 [==============================] - 0s 21ms/step
2022   1
1/1 [==============================] - 0s 21ms/step
2023   1
1/1 [==============================] - 0s 21ms/step
2024   1
1/1 [==============================] - 0s 20ms/step
2025   1
1/1 [==============================] - 0s 41ms/step
2026   1
1/1 [==============================] - 0s 33ms/step
2027   1
1/1 [===================

In [ ]:
entlistindex

100

In [ ]:
newdata

['Entity,Year,(% equivalent primary energy)',
 'Afghanistan,2020,1.3612135618925094',
 'Afghanistan,2021,1.3437928152084349',
 'Afghanistan,2022,1.3131672613648002',
 'Afghanistan,2023,1.32543552084671',
 'Afghanistan,2024,1.325022415262465',
 'Afghanistan,2025,1.3429744364555583',
 'Afghanistan,2026,1.3363645118080756',
 'Afghanistan,2027,1.3329441233800599',
 'Africa (UN),2020,5.533722217082977',
 'Africa (UN),2021,5.548136440515519',
 'Africa (UN),2022,5.537425028085709',
 'Africa (UN),2023,5.558598049124914',
 'Africa (UN),2024,5.547025850436418',
 'Africa (UN),2025,5.5484669116525565',
 'Africa (UN),2026,5.5512640814054235',
 'Africa (UN),2027,5.550567572818801',
 'Algeria,2020,8.749097153544426',
 'Algeria,2021,8.775926977396011',
 'Algeria,2022,8.735883942246437',
 'Algeria,2023,8.773905381560326',
 'Algeria,2024,8.756168420640943',
 'Algeria,2025,8.763602146674023',
 'Algeria,2026,8.76876439049619',
 'Algeria,2027,8.763479296965677',
 'Andorra,2020,23.960441460609438',
 'Andorr

In [ ]:
import csv
with open('predictions.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in newdata:
        csvwriter.writerow(row.split(','))